In [1]:
#Load libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer
import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from scipy import stats
from sklearn.cluster import KMeans
from joblib import dump
import geohash2 as geohash
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
from torch.optim.lr_scheduler import LambdaLR
import torch.nn.functional as F


In [2]:
# Load the dataset from csv file
df = pd.read_csv('onetoothotfeaturesleft68.csv', low_memory=False)

Yolo-Modeling

In [3]:
# Update the list with all the columns now that they have been one-hot encoded
bool_columns_list = [col for col in df if col != 'Amount']
# Convert boolean columns to float
df[bool_columns_list] = df[bool_columns_list].astype(float)

# Define your features (X) and target (y) from the DataFrame
X = df[bool_columns_list]
y = df['Amount']

# Splitting the dataset into training and test sets (as you've done)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

# Further split the training set into a new training set and a validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

# Converting to PyTorch tensors and moving to the specified device
device = torch.device('cuda')

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).to(device)
X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).to(device)

# Creating datasets and dataloaders for training, validation, and test sets
train_data = TensorDataset(X_train_tensor, y_train_tensor)
val_data = TensorDataset(X_val_tensor, y_val_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_data, batch_size=750, shuffle=True)
val_loader = DataLoader(val_data, batch_size=750, shuffle=False)
test_loader = DataLoader(test_data, batch_size=750, shuffle=False)


In [4]:
device = torch.device("cuda")
print(device)

cuda


In [5]:
class RegressionModel(nn.Module):
    def __init__(self, input_size):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)  # Dynamically set input size
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x


In [6]:
# Initialize the model with the correct input size
input_size = X_train_tensor.shape[1]
model = RegressionModel(input_size).to(device)

In [7]:
# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.005, betas=(0.9, 0.999), eps=1e-08)

# Define a lambda function for learning rate decay
def lr_lambda(epoch):
    initial_lr = 0.01
    final_lr = 0.001
    max_epochs = 200
    decay_rate = (final_lr / initial_lr) ** (1 / max_epochs)
    return decay_rate ** epoch

scheduler = LambdaLR(optimizer, lr_lambda)

In [8]:
# Training the model
# Optimizer: Adam with specified parameters and a learning rate of 0.001

n_epochs = 200
early_stop_threshold = 0.99
for epoch in range(n_epochs):
    
    model.train()
    train_loss = 0.0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)  # Move data to the device
        
        # Clear the gradients of all optimized tensors
        optimizer.zero_grad()
        
        # Forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        
        # Calculate the loss
        loss = criterion(output.view(-1), target)
        
        # Backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        
        # Perform a single optimization step (parameter update)
        optimizer.step()
        scheduler.step()
        # Update running training loss
        train_loss += loss.item() * data.size(0)
    
    model.eval()
    with torch.no_grad():
        val_predictions = []
        val_targets = []
        for X, y in val_loader:
            X, y = X.to(device), y.to(device)
            outputs = model(X)
            val_predictions.extend(outputs.view(-1).cpu().numpy())
            val_targets.extend(y.cpu().numpy())
        
        val_r2_score = r2_score(val_targets, val_predictions)
        print(f'Epoch {epoch+1}/{n_epochs}, Validation R2 Score: {val_r2_score}')

        if val_r2_score >= early_stop_threshold:
            print(f"Early stopping at epoch {epoch+1} with R2 Score: {val_r2_score}")
            break

Epoch 1/200, Validation R2 Score: 0.3435581644430664
Epoch 2/200, Validation R2 Score: 0.35917649941981744
Epoch 3/200, Validation R2 Score: 0.36227305151410805
Epoch 4/200, Validation R2 Score: 0.3638010422172152
Epoch 5/200, Validation R2 Score: 0.36396527912252896
Epoch 6/200, Validation R2 Score: 0.364085446865428
Epoch 7/200, Validation R2 Score: 0.3640872224838416
Epoch 8/200, Validation R2 Score: 0.3640858574749569
Epoch 9/200, Validation R2 Score: 0.3640860985247425
Epoch 10/200, Validation R2 Score: 0.36408608371135853
Epoch 11/200, Validation R2 Score: 0.36408608341177207
Epoch 12/200, Validation R2 Score: 0.36408608334606907
Epoch 13/200, Validation R2 Score: 0.36408608336406856
Epoch 14/200, Validation R2 Score: 0.36408608341128024
Epoch 15/200, Validation R2 Score: 0.36408608341128024
Epoch 16/200, Validation R2 Score: 0.36408608341128024
Epoch 17/200, Validation R2 Score: 0.36408608341128024
Epoch 18/200, Validation R2 Score: 0.36408608341128024
Epoch 19/200, Validation R

In [9]:
# Evaluating the model in detail
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor).view(-1)
    
    # Convert tensors to numpy arrays for compatibility with sklearn metrics
    y_test_np = y_test_tensor.cpu().numpy()
    predictions_np = predictions.cpu().numpy()

    # Calculating MAE
    mae = mean_absolute_error(y_test_np, predictions_np)
    print(f'MAE: {mae}')

    # RMSE (already calculated)
    mse = criterion(predictions, y_test_tensor)
    rmse = torch.sqrt(mse)
    print(f'RMSE: {rmse.item()}')

    # R2 Score (already calculated)
    r2 = r2_score(y_test_np, predictions_np)
    print(f'R2 Score: {r2}')

    # Calculating MAPE
    mape = np.mean(np.abs((y_test_np - predictions_np) / y_test_np)) * 100
    print(f'MAPE: {mape}')

    # Adjusted R2 (for multiple regression models)
    n = len(y_test_np) # Number of samples
    p = X_test_tensor.shape[1] # Number of independent variables
    adjusted_r2 = 1 - (1-r2)*(n-1)/(n-p-1)
    print(f'Adjusted R2 Score: {adjusted_r2}')

    print("---------First 10 Predictions---------")
    for actual, predicted in zip(y_test_np[:10], predictions_np[:10]):
        absolute_error = np.abs(actual - predicted)
        relative_error = (absolute_error / actual) * 100 if actual != 0 else float('inf')
        print(f"Actual/Predicted: {actual}/{predicted}")
        print(f"Abs Error: {absolute_error}")
        print(f"Rel Error: {relative_error:.2f}%")

MAE: 151.5369873046875
RMSE: 214.5843048095703
R2 Score: 0.3787584380812301
MAPE: inf
Adjusted R2 Score: 0.37659888714999845
---------First 10 Predictions---------
Actual/Predicted: 599.0/748.9151000976562
Abs Error: 149.91510009765625
Rel Error: 25.03%
Actual/Predicted: 435.0/462.0944519042969
Abs Error: 27.094451904296875
Rel Error: 6.23%
Actual/Predicted: 1523.0/822.33154296875
Abs Error: 700.66845703125
Rel Error: 46.01%
Actual/Predicted: 648.5614624023438/495.00518798828125
Abs Error: 153.5562744140625
Rel Error: 23.68%
Actual/Predicted: 845.0/838.6952514648438
Abs Error: 6.30474853515625
Rel Error: 0.75%
Actual/Predicted: 376.0/433.3973388671875
Abs Error: 57.3973388671875
Rel Error: 15.27%
Actual/Predicted: 416.0/423.47412109375
Abs Error: 7.47412109375
Rel Error: 1.80%
Actual/Predicted: 899.0/678.4435424804688
Abs Error: 220.55645751953125
Rel Error: 24.53%
Actual/Predicted: 648.5614624023438/773.7118530273438
Abs Error: 125.150390625
Rel Error: 19.30%
Actual/Predicted: 951.429

C:\Users\Tono\AppData\Local\Temp\ipykernel_11692\3384580297.py:26: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test_np - predictions_np) / y_test_np)) * 100


In [10]:
# save model
#torch.save(model, 'Amazon.pth')